In [1]:
# code for STA/LTA detections 
using Plots
using Statistics

#sample data object
struct sampleData
    trace
    sampleRate
    startTime
end

#make sample data - random signal centered on 0
sample = sampleData(rand(10000).- 0.5,10,0)

#add an impulsive spike
sample.trace[9980] = 2.5
sample.trace[9982] = 2.5
sample.trace[9984] = 2.5
sample.trace[9986] = 2.5
sample.trace[9988] = 2.5
sample.trace[9990] = 2.5
sample.trace[9992] = 2.5
sample.trace[9994] = 2.5
sample.trace[9996] = 2.5
sample.trace[9998] = 2.5

sample.trace[9981] = -2.5
sample.trace[9983] = -2.5
sample.trace[9985] = -2.5
sample.trace[9987] = -2.5
sample.trace[9989] = -2.5
sample.trace[9991] = -2.5
sample.trace[9993] = -2.5
sample.trace[9995] = -2.5
sample.trace[9997] = -2.5
sample.trace[9999] = -2.5


function detect(data,longWinLength::Int=20,shortWinLength::Int=2,threshold::Int=2,overlap::Int=10)
    
    #error messages to add
    #window must be shorter than data length
    #short window must be shorter than long window
    #threshold must be greater than 1

    #convert window lengths from seconds to samples
    longWinLength = longWinLength * data.sampleRate
    shortWinLength = shortWinLength * data.sampleRate
    overlap = overlap * data.sampleRate
    
    #calculate how much to move beginning of window each iteration
    slide = longWinLength-overlap
    
    numShortWin = longWinLength / shortWinLength
    detections = Float32[]
    
    
    #loop through data in long windows
    t = 0
    i = 0
    while i < length(data.trace)
        
        #check if last window
        if length(data.trace) - i <= longWinLength
            longWinLength = length(data.trace)-i
            print("trig")
            print(longWinLength)
            i = i * 10000

        end
        
        longTrace = data.trace[i+1:i+longWinLength]
        lta = mean( broadcast(abs,longTrace))

        #loop through long window in short windows 
        n = 0
        while n < longWinLength - shortWinLength
            shortTrace = data.trace[i+n+1:i+n+shortWinLength]
            sta = mean(broadcast(abs,shortTrace))
            staLta = sta / lta        

            #record detection time if trigger condition is met
            if staLta > threshold
                trigger = data.startTime + (i+n+1)/data.sampleRate
                detections = [detections;trigger]

            end
            
            #advance short window
            n = n + shortWinLength
        end
        #advance window
        i = i + slide

    end  
    
    return unique(detections)
    
end

detections = detect(sample)
print(detections)
plot(sample.trace)


trig200

BoundsError: BoundsError: attempt to access 10000-element Array{Float64,1} at index [98000001:98000200]